In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [51]:
def met_gradiente(A, x0, maxiter = 100000):
    trayectoria = [x0]
    f = lambda x: 0.5*x.T@A@x
    x = x0.copy()
    for _ in range(maxiter):
        gradf = A@x
        l = gradf.T@gradf/(gradf.T@A@gradf)
        x -= l*gradf
        trayectoria += [x.copy()]
    return x, trayectoria

In [52]:
x0 = np.random.rand(3)*10 - 5
x0

array([ 0.93459244, -1.86154692, -1.69430564])

In [53]:
A = np.diag(np.random.rand(3))*100

In [54]:
xf, trayectoria = met_gradiente(A, x0)

In [55]:
trayectoria

[array([ 0.93459244, -1.86154692, -1.69430564]),
 array([ 0.12341067,  0.18421776, -0.91646109]),
 array([-0.05697324, -0.15671   , -0.20792128]),
 array([-0.00457847,  0.02576339, -0.1067817 ]),
 array([ 0.0018273 , -0.01987601, -0.02775877]),
 array([ 0.0001332 ,  0.00345554, -0.01414639]),
 array([-5.26164500e-05, -2.64797332e-03, -3.70811430e-03]),
 array([-3.81479502e-06,  4.61682938e-04, -1.88895279e-03]),
 array([ 1.50615816e-06, -3.53674108e-04, -4.95332458e-04]),
 array([ 1.09171992e-07,  6.16724028e-05, -2.52322845e-04]),
 array([-4.31023461e-08, -4.72437085e-05, -6.61667771e-05]),
 array([-3.12418369e-09,  8.23823579e-06, -3.37053926e-05]),
 array([ 1.23346204e-09, -6.31083810e-06, -8.83859335e-06]),
 array([ 8.94048771e-11,  1.10046795e-06, -4.50238413e-06]),
 array([-3.52980259e-11, -8.43005106e-07, -1.18066401e-06]),
 array([-2.55850240e-12,  1.47001094e-07, -6.01430870e-07]),
 array([ 1.01012480e-12, -1.12609071e-07, -1.57713727e-07]),
 array([ 7.32167498e-14,  1.9636484